In [1]:
import geopandas as gpd
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import os
from tqdm import tqdm
import IPython.display as display
import copy
import seaborn as sns
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler
import matplotlib.cm as cm
import plotly.io as pio
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize

/opt/anaconda3/envs/geospatial/lib/python3.8/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/var/folders/1k/27mkp8bj3ps60c3nmr7rbqzh0000gn/T/ipykernel_20278/4162652126.py:1: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch shou

In [2]:
def merge_csv_files(directory):
    # Get a list of all the csv files
    csv_files = [f for f in os.listdir(directory) if f.endswith('.csv')]

    # Initialize an empty list to hold dataframes
    dfs = []

    # Loop through csv files, read each into a dataframe, and append to the list
    for file in csv_files:
        # Extract date from filename, assuming the date is in format 'traffic_flow_YYYY_MM_DD'
        date_str = file.split('.')[0].split('_')[-3:]  # This gives ['YYYY', 'MM', 'DD']
        date = datetime.strptime('_'.join(date_str), '%Y_%m_%d').date()

        df = pd.read_csv(os.path.join(directory, file))

        # Add date as a new column
        df['date'] = date.strftime('%m/%d/%y')

        dfs.append(df)

    # Concatenate all dataframes in the list into one dataframe
    merged_df = pd.concat(dfs, ignore_index=True).drop_duplicates()

    # Return the merged dataframe
    return merged_df


traffic_flows = merge_csv_files(
    '/Users/zonghe/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Zonghe Ma/Raw data/[XH]Traffic flow')
road_network = '/Users/zonghe/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Zonghe Ma/Raw data/[XH]road_network/road_network.shp'

# clean the traffic flow data
traffic_flows = traffic_flows.drop_duplicates(['toid', 'date'])
traffic_flows = traffic_flows.groupby(['toid', 'date']).agg(
    {'bus': 'sum', 'car': 'sum', 'cycle': 'sum', 'walks': 'sum', 'stationary': 'sum'}).reset_index()

lsoa = '/Users/zonghe/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Zonghe Ma/Raw data/London administrative boundaries/london_LSOA/london_LSOA.shp'

inoutter = '/Users/zonghe/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Zonghe Ma/Raw data/London administrative boundaries/lp-consultation-oct-2009-inner-outer-london-shp/lp-consultation-oct-2009-inner-outer-london.shp'
# tube_line = 'https://raw.githubusercontent.com/oobrien/vis/master/tubecreature/data/tfl_lines.json'
# tube_station = 'https://raw.githubusercontent.com/oobrien/vis/master/tubecreature/data/tfl_stations.json'

inoutter = gpd.read_file(inoutter)
inoutter.to_crs(epsg=27700, inplace=True)

# tube_station = gpd.read_file(tube_station)
# tube_station.to_crs(epsg=27700, inplace=True)
# tube_station = gpd.sjoin(tube_station, inoutter, op='within')

# tube_line = gpd.read_file(tube_line)
# tube_line.to_crs(epsg=27700, inplace=True)
# tube_line = gpd.sjoin(tube_line, inoutter, op='within')

lsoa = gpd.read_file(lsoa, crs={'init': 'epsg:27700'})
road_network = gpd.read_file(road_network, crs={'init': 'epsg:27700'})
road_network.rename(columns={'NAME': 'boroughs'}, inplace=True)
road_network.loc[:, ['cycle_lane', 'bus_lane']] = road_network[['cycle_lane', 'bus_lane']].fillna('n')


# clean the traffic flow data
traffic_flows = traffic_flows.drop_duplicates(['toid', 'date'])
traffic_flows = traffic_flows.groupby(['toid', 'date']).agg(
    {'bus': 'sum', 'car': 'sum', 'cycle': 'sum', 'walks': 'sum', 'stationary': 'sum'}).reset_index()
traffic_flows['total'] = traffic_flows['bus'] + traffic_flows['car'] + traffic_flows['cycle'] + traffic_flows[
    'walks'] + traffic_flows['stationary']

flows = pd.merge(
    road_network[
        ['toid', 'roadclassi', 'geometry', 'cycle_lane', 'bus_lane', 'boroughs']],
    traffic_flows, left_on='toid', right_on='toid', how='left')
flows.set_geometry('geometry', inplace=True)

flows['classification'] = flows['roadclassi'].replace(
    {'Unknown': 'Local Road', 'Not Classified': 'Local Road', 'Unclassified': 'Local Road',
     'Classified Unnumbered': 'Local Road', 'A Road': 'Strategic Road', 'B Road': 'Strategic Road'})

flows.drop(columns=['roadclassi'], inplace=True)

stage_date = ['03/01/22', '02/22/22', '03/08/22']
flows = flows.loc[flows['date'].isin(stage_date)]

# label the regional level
flows = gpd.sjoin(flows, inoutter, how='inner', predicate='within')
flows = flows.drop(columns=['index_right', 'Source', 'Area_Ha', 'Shape_Leng', 'Shape_Area'])
flows.reset_index(drop=True, inplace=True)


merged = flows
flows = merged

# convert the dataframe
flows = pd.melt(flows,
                id_vars=['toid', 'classification', 'geometry', 'date', 'Boundary', 'cycle_lane', 'bus_lane',
                         'boroughs'],
                var_name='mode', value_name='flow')

flows = pd.pivot_table(flows,
                       index=['toid', 'classification', 'geometry', 'Boundary', 'mode', 'cycle_lane', 'bus_lane',
                              'boroughs'],
                       columns='date',
                       values='flow',
                       aggfunc='first').reset_index()


flows = flows.groupby(
    ['toid', 'mode', 'classification', 'geometry', 'Boundary', 'cycle_lane', 'bus_lane', 'boroughs'],
    as_index=False).agg(
    {'03/01/22': 'first', '02/22/22': 'first', '03/08/22': 'first'})
# Calculate the impact and recovery flows for one strike
flows['impact_flow'] = flows['03/01/22'] - flows['02/22/22']
flows['recovery_flow'] = flows['03/08/22'] - flows['03/01/22']

# Calculate impact rate while avoiding division by zero
flows['impact_rate'] = flows.apply(
    lambda row: round(row['impact_flow'] / row['02/22/22'], 4) if row['02/22/22'] != 0 else 0, axis=1)
# Calculate recovery rate while avoiding division by zero
flows['recovery_rate'] = flows.apply(
    lambda row: round(row['recovery_flow'] / row['03/01/22'], 4) if row['03/01/22'] != 0 else 0, axis=1)

All = flows.copy()
All

date,toid,mode,classification,geometry,Boundary,cycle_lane,bus_lane,boroughs,03/01/22,02/22/22,03/08/22,impact_flow,recovery_flow,impact_rate,recovery_rate
0,osgb4000000027865921,bus,Motorway,"LINESTRING (531539.442 200769.874, 531592.988 ...",Outer London,n,n,Enfield,16,11,12,5,-4,0.4545,-0.2500
1,osgb4000000027865921,car,Motorway,"LINESTRING (531539.442 200769.874, 531592.988 ...",Outer London,n,n,Enfield,1041,1100,1081,-59,40,-0.0536,0.0384
2,osgb4000000027865921,cycle,Motorway,"LINESTRING (531539.442 200769.874, 531592.988 ...",Outer London,n,n,Enfield,14,4,7,10,-7,2.5000,-0.5000
3,osgb4000000027865921,stationary,Motorway,"LINESTRING (531539.442 200769.874, 531592.988 ...",Outer London,n,n,Enfield,2,0,1,2,-1,0.0000,-0.5000
4,osgb4000000027865921,total,Motorway,"LINESTRING (531539.442 200769.874, 531592.988 ...",Outer London,n,n,Enfield,1095,1151,1122,-56,27,-0.0487,0.0247
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1742635,osgb5000005242182149,car,Local Road,"LINESTRING (543548.945 179236.408, 543554.000 ...",Inner London,n,n,Greenwich,33,54,51,-21,18,-0.3889,0.5455
1742636,osgb5000005242182149,cycle,Local Road,"LINESTRING (543548.945 179236.408, 543554.000 ...",Inner London,n,n,Greenwich,2,1,1,1,-1,1.0000,-0.5000
1742637,osgb5000005242182149,stationary,Local Road,"LINESTRING (543548.945 179236.408, 543554.000 ...",Inner London,n,n,Greenwich,3,4,5,-1,2,-0.2500,0.6667
1742638,osgb5000005242182149,total,Local Road,"LINESTRING (543548.945 179236.408, 543554.000 ...",Inner London,n,n,Greenwich,50,74,70,-24,20,-0.3243,0.4000


In [3]:

flow_change = All.copy()
flow_change.drop(
    columns={'toid', 'geometry', 'impact_flow', 'recovery_flow', 'impact_rate', 'recovery_rate'},
    inplace=True)

flow_change = flow_change.groupby(['mode', 'classification', 'Boundary', 'boroughs']).agg(
    {'03/01/22': 'sum', '02/22/22': 'sum', '03/08/22': 'sum', 'cycle_lane': 'first',
     'bus_lane': 'first'}).reset_index().rename_axis(None, axis=1)
flow_change = flow_change.astype({'03/01/22': int, '02/22/22': int, '03/08/22': int})
flow_change = flow_change[flow_change['mode'] != 'total']
flow_change.insert(0, 'Total Flows', 'Total Flows')
flow_change['impact_flow'] = flow_change['03/01/22'] - flow_change['02/22/22']
flow_change['recovery_flow'] = flow_change['03/08/22'] - flow_change['03/01/22']

# Calculate impact rate while avoiding division by zero
flow_change['impact_rate'] = flow_change.apply(
    lambda row: round(row['impact_flow'] / row['02/22/22'], 4) if row['02/22/22'] != 0 else 0, axis=1)
flow_change['recovery_rate'] = flow_change.apply(
    lambda row: round(row['recovery_flow'] / row['03/01/22'], 4) if row['03/01/22'] != 0 else 0, axis=1)

# 获取所有列的列表
columns = flow_change.columns

# 遍历每列，将内容转换为首字母大写
for column in columns:
    if flow_change[column].dtype == 'object':  # 仅对字符串列进行操作
        flow_change[column] = flow_change[column].str.title()  # 使用str.title()函数将首字母大写

# 获取除了非数值列（例如日期和字符串）之外的所有列
numeric_columns = flow_change.select_dtypes(include=['number']).columns

# 创建MinMaxScaler对象
scaler = MinMaxScaler()

# 使用fit_transform方法对数值列进行缩放
flow_change[numeric_columns] = scaler.fit_transform(flow_change[numeric_columns])

flow_change

,Total Flows,mode,classification,Boundary,boroughs,03/01/22,02/22/22,03/08/22,cycle_lane,bus_lane,impact_flow,recovery_flow,impact_rate,recovery_rate
0,Total Flows,Bus,Local Road,Inner London,Camden,0.010747,0.022021,0.022628,N,N,0.785132,0.288018,0.166167,0.901885
1,Total Flows,Bus,Local Road,Inner London,City Of London,0.005768,0.009165,0.010165,N,N,0.822392,0.245652,0.214267,0.743829
2,Total Flows,Bus,Local Road,Inner London,Greenwich,0.013020,0.014118,0.013313,N,N,0.834164,0.220209,0.314000,0.402842
3,Total Flows,Bus,Local Road,Inner London,Hackney,0.008178,0.008614,0.010117,N,N,0.836835,0.230842,0.323233,0.501770
4,Total Flows,Bus,Local Road,Inner London,Hammersmith And Fulham,0.007074,0.007902,0.008053,N,N,0.834845,0.225510,0.304800,0.456276
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451,Total Flows,Walks,Strategic Road,Outer London,Merton,0.041204,0.042459,0.043669,N,N,0.836309,0.226583,0.330433,0.420015
452,Total Flows,Walks,Strategic Road,Outer London,Redbridge,0.035988,0.040284,0.038932,N,N,0.821186,0.230525,0.304167,0.430157
453,Total Flows,Walks,Strategic Road,Outer London,Richmond Upon Thames,0.044425,0.049097,0.049141,N,N,0.820258,0.238942,0.308100,0.441351
454,Total Flows,Walks,Strategic Road,Outer London,Sutton,0.036153,0.038690,0.037542,N,N,0.829642,0.221453,0.318167,0.410161


In [ ]:
total = ['Total Flows']
modes = ['Bus', 'Car', 'Cycle', 'Walks', 'Stationary']
boundary_nodes = ['Motorway', 'Strategic Road', 'Local Road', 'Inner London', 'Outer London']

nodes = total + modes + boundary_nodes

node_indices = {node: index for index, node in enumerate(nodes)}

dates = ['03/01/22', '02/22/22', '03/08/22']

# # 设置内置主题
pio.templates.default = 'plotly'  # 可以更改为其他内置主题名称 ['plotly'：默认主题。'simple_white'：简洁的白色背景主题。'ggplot2'：仿照ggplot2的主题。'seaborn'：仿照seaborn的主题。'plotly_dark'：深色背景主题。]


# 设置节点颜色，现有流量颜色与目标节点颜色相同
# node_colors = ['blue', 'green', 'orange', 'red', 'purple', 'cyan', 'gray', 'pink', 'brown', 'yellow', 'magenta', 'teal']

# 使用 matplotlib 的颜色映射生成节点颜色

def convert_to_rgba(color, alpha):
    return f'rgba({int(color[0] * 255)}, {int(color[1] * 255)}, {int(color[2] * 255)}, {alpha})'


cmap = cm.get_cmap(
    # 'twilight'
    'tab20c'
    , len(nodes))  # 选择一个颜色映射
node_colors = [convert_to_rgba(cmap(i), 0.6) for i in range(len(nodes))]  # 生成对应数量的颜色

for date in dates:
    sankey_data = []
    for i, row in flow_change.iterrows():
        source_node = row['Total Flows']
        target_node = row['mode']
        value = row[date]

        sankey_data.append({
            'source': node_indices[source_node],
            'target': node_indices[target_node],
            'value': value,
            'color': node_colors[node_indices[source_node]]
        })

        source_node = row['mode']
        target_node = row['classification']
        value = row[date]

        sankey_data.append({
            'source': node_indices[source_node],
            'target': node_indices[target_node],
            'value': value,
            'color': node_colors[node_indices[source_node]]
        })

        source_node = row['classification']
        target_node = row['Boundary']
        value = row[date]

        sankey_data.append({
            'source': node_indices[source_node],
            'target': node_indices[target_node],
            'value': value,
            'color': node_colors[node_indices[source_node]]
        })

    fig = go.Figure(go.Sankey(

        arrangement='freeform',

        node=dict(
            pad=10,
            thickness=20,
            line=dict(color='black', width=0.3),
            label=nodes,
            color=node_colors
        ),
        link=dict(
            source=[link['source'] for link in sankey_data],
            target=[link['target'] for link in sankey_data],
            value=[link['value'] for link in sankey_data],
            color=[link['color'] for link in sankey_data],
        ),

    ))

    fig.update_layout(
        title_text=f"Road Space Reallocation on {date}",
        font_size=15,
        autosize=True,
        hovermode='closest'
    )

    fig.show()


/var/folders/1k/27mkp8bj3ps60c3nmr7rbqzh0000gn/T/ipykernel_20278/1491706607.py:24: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = cm.get_cmap(


In [6]:
class_mode_t = All.copy()

class_mode_t = class_mode_t.groupby(['mode', 'classification', 'Boundary']).agg(
    {'03/01/22': 'sum', '02/22/22': 'sum', '03/08/22': 'sum', 'cycle_lane': 'first',
     'bus_lane': 'first'}).reset_index().rename_axis(None, axis=1)
class_mode_t = class_mode_t.astype({'03/01/22': int, '02/22/22': int, '03/08/22': int})

class_mode_t['impact_flow'] = class_mode_t['03/01/22'] - class_mode_t['02/22/22']
class_mode_t['recovery_flow'] = class_mode_t['03/08/22'] - class_mode_t['03/01/22']
# Calculate impact rate while avoiding division by zero
class_mode_t['impact_rate'] = class_mode_t.apply(
    lambda row: round(row['impact_flow'] / row['02/22/22'], 4) if row['02/22/22'] != 0 else 0, axis=1)
# Calculate recovery rate while avoiding division by zero
class_mode_t['recovery_rate'] = class_mode_t.apply(
    lambda row: round(row['recovery_flow'] / row['03/01/22'], 4) if row['03/01/22'] != 0 else 0, axis=1)

# 获取所有列的列表
columns = class_mode_t.columns

# 遍历每列，将内容转换为首字母大写
for column in columns:
    if class_mode_t[column].dtype == 'object':  # 仅对字符串列进行操作
        class_mode_t[column] = class_mode_t[column].str.title()  # 使用str.title()函数将首字母大写

class_mode_t = class_mode_t.pivot_table(index='classification', columns='mode', values=['impact_rate', 'recovery_rate'])
# 将列名重新整理成多重索引的形式
class_mode_t.columns = [f'{col[1]}/{col[0]}' for col in class_mode_t.columns]

class_mode_t

,Bus/impact_rate,Car/impact_rate,Cycle/impact_rate,Stationary/impact_rate,Total/impact_rate,Walks/impact_rate,Bus/recovery_rate,Car/recovery_rate,Cycle/recovery_rate,Stationary/recovery_rate,Total/recovery_rate,Walks/recovery_rate
classification,,,,,,,,,,,,
Local Road,-0.14575,-0.06745,0.1694,-0.04405,-0.06820,-0.09825,0.2167,0.05670,-0.10805,0.0332,0.05745,0.08135
Motorway,-0.13560,-0.01090,0.3227,-0.02110,-0.01430,-0.15880,0.4031,-0.03340,-0.33980,-0.1828,-0.02730,0.12710
Strategic Road,-0.14255,-0.06910,0.1943,-0.00560,-0.06605,-0.08695,0.2123,0.05295,-0.13425,-0.0296,0.04680,0.03785


In [12]:
class_mode_t2 = All.copy()

class_mode_t2 = class_mode_t2.groupby(['mode', 'classification', 'Boundary', 'boroughs']).agg(
    {'03/01/22': 'sum', '02/22/22': 'sum', '03/08/22': 'sum', 'cycle_lane': 'first',
     'bus_lane': 'first'}).reset_index().rename_axis(None, axis=1)
class_mode_t2 = class_mode_t2.astype({'03/01/22': int, '02/22/22': int, '03/08/22': int})

class_mode_t2['impact_flow'] = class_mode_t2['03/01/22'] - class_mode_t2['02/22/22']
class_mode_t2['recovery_flow'] = class_mode_t2['03/08/22'] - class_mode_t2['03/01/22']
# Calculate impact rate while avoiding division by zero
class_mode_t2['impact_rate'] = class_mode_t2.apply(
    lambda row: round(row['impact_flow'] / row['02/22/22'], 4) if row['02/22/22'] != 0 else 0, axis=1)
# Calculate recovery rate while avoiding division by zero
class_mode_t2['recovery_rate'] = class_mode_t2.apply(
    lambda row: round(row['recovery_flow'] / row['03/01/22'], 4) if row['03/01/22'] != 0 else 0, axis=1)

# 获取所有列的列表
columns = class_mode_t2.columns

# 遍历每列，将内容转换为首字母大写
for column in columns:
    if class_mode_t2[column].dtype == 'object':  # 仅对字符串列进行操作
        class_mode_t2[column] = class_mode_t2[column].str.title()  # 使用str.title()函数将首字母大写

class_mode_t2 = class_mode_t2.pivot_table(index=['boroughs','classification'], columns='mode', values=['impact_rate', 'recovery_rate'])
# 将列名重新整理成多重索引的形式
# class_mode_t2.columns = [f'{col[1]}/{col[0]}' for col in class_mode_t2.columns]
# class_mode_t2.index = [f'{col[0]}/{col[1]}' for col in class_mode_t2.index]

class_mode_t2

impact_rate                             \
mode                                        Bus     Car   Cycle Stationary   
boroughs             classification                                          
Barking And Dagenham Local Road          0.0383 -0.0596  0.0407     0.0236   
                     Strategic Road     -0.0525 -0.0377  0.0759     0.0483   
Barnet               Local Road          0.0254 -0.0510  0.1292    -0.0364   
                     Motorway           -0.1742  0.1433  0.6695    -0.6923   
                     Strategic Road      0.0310  0.0112  0.1662    -0.0750   
...                                         ...     ...     ...        ...   
Waltham Forest       Strategic Road     -0.0751 -0.0649  0.1702     0.0840   
Wandsworth           Local Road         -0.1822 -0.0771  0.1400     0.0168   
                     Strategic Road     -0.1976 -0.0743  0.1905    -0.0515   
Westminster          Local Road         -0.5691 -0.2351  0.3765    -0.1809   
                     Strategic Road     -0.5567 -0.2719  0.2563    -0.1194   

                                                    recovery_rate          \
mode                                  Total   Walks           Bus     Car   
boroughs             classification                                         
Barking And Dagenham Local Road     -0.0447  0.0295        0.0224  0.0880   
                     Strategic Road -0.0327  0.0135        0.0672  0.0350   
Barnet               Local Road     -0.0589 -0.1726        0.0473  0.0078   
                     Motorway        0.1313 -0.0359        0.5806 -0.0523   
                     Strategic Road -0.0003 -0.1942        0.0676 -0.0134   
...                                     ...     ...           ...     ...   
Waltham Forest       Strategic Road -0.0525  0.0207        0.0383  0.0451   
Wandsworth           Local Road     -0.0806 -0.1286        0.4341  0.0655   
                     Strategic Road -0.0747 -0.1188        0.4517  0.0395   
Westminster          Local Road     -0.2431 -0.2759        1.2333  0.2236   
                     Strategic Road -0.2564 -0.2130        1.1208  0.2819   

                                                                        
mode                                  Cycle Stationary   Total   Walks  
boroughs             classification                                     
Barking And Dagenham Local Road     -0.2668    -0.0068  0.0689  0.0099  
                     Strategic Road -0.1882    -0.0118  0.0307  0.0186  
Barnet               Local Road     -0.2742     0.0369  0.0135  0.1229  
                     Motorway       -0.2420     0.2500 -0.0455 -0.0790  
                     Strategic Road -0.2540     0.0074 -0.0108  0.0904  
...                                     ...        ...     ...     ...  
Waltham Forest       Strategic Road -0.0802    -0.0905  0.0327 -0.0490  
Wandsworth           Local Road     -0.0303    -0.0102  0.0742  0.1017  
                     Strategic Road -0.0658    -0.0145  0.0461  0.0589  
Westminster          Local Road     -0.1848     0.1372  0.2004  0.1319  
                     Strategic Road -0.2005     0.0043  0.2170  0.0225  

[74 rows x 12 columns]

In [13]:
class_mode_f = All.copy()
# 获取所有列的列表
columns = class_mode_f.columns

# 遍历每列，将内容转换为首字母大写
for column in columns:
    if class_mode_f[column].dtype == 'object':  # 仅对字符串列进行操作
        class_mode_f[column] = class_mode_f[column].str.title()  # 使用str.title()函数将首字母大写

class_mode_f = class_mode_f.pivot_table(index=['classification', 'toid'], values=['impact_rate', 'recovery_rate'],
                                        columns='mode')
# 将列名重新整理成多重索引的形式
# class_mode_f.columns = [f'{col[1]}/{col[0]}' for col in class_mode_f.columns]
class_mode_f = class_mode_f.swaplevel(axis=1)

# 定义第一层和第二层索引的顺序
first_level_order = ['Total', 'Car', 'Bus', 'Cycle', 'Walks', 'Stationary']
second_level_order = ['impact_rate', 'recovery_rate']

# 初始化一个空列表用于存储排序后的列名
sorted_columns = []

# 循环遍历第一层索引的顺序
for first_level in first_level_order:
    # 循环遍历第二层索引的顺序
    for second_level in second_level_order:
        # 构建当前列名
        current_column = (first_level, second_level)
        # 将当前列名添加到排序后的列名列表中
        sorted_columns.append(current_column)

# 使用排序后的列名对 DataFrame 进行排序
class_mode_f = class_mode_f[sorted_columns]

class_mode_f


mode                                      Total                       Car  \
date                                impact_rate recovery_rate impact_rate   
classification toid                                                         
Local Road     Osgb4000000027869139      0.0416        0.1171      0.0403   
               Osgb4000000027869149      0.0000        0.0000      0.0000   
               Osgb4000000027876087     -1.0000        0.0000     -1.0000   
               Osgb4000000027876088     -0.1105        0.1893     -0.1047   
               Osgb4000000027876095      0.1910        0.0188      0.2046   
...                                         ...           ...         ...   
Strategic Road Osgb5000005241746776     -0.0168       -0.0546     -0.0504   
               Osgb5000005241746777      0.0000        0.0000      0.0000   
               Osgb5000005241864717     -0.0963        0.0492     -0.0577   
               Osgb5000005242071802      0.2846        0.4051      0.2212   
               Osgb5000005242156433     -0.1752        0.0490     -0.1510   

mode                                                      Bus                \
date                                recovery_rate impact_rate recovery_rate   
classification toid                                                           
Local Road     Osgb4000000027869139        0.1108     -1.0000           0.0   
               Osgb4000000027869149        0.0000      0.0000           0.0   
               Osgb4000000027876087        0.0000      0.0000           0.0   
               Osgb4000000027876088        0.2532      0.0000          -0.5   
               Osgb4000000027876095        0.0174      0.2000          -1.0   
...                                           ...         ...           ...   
Strategic Road Osgb5000005241746776       -0.0076      4.0000          -0.2   
               Osgb5000005241746777        0.0000      0.0000           0.0   
               Osgb5000005241864717        0.0510     -0.5000           1.0   
               Osgb5000005242071802        0.3188      0.3333           0.5   
               Osgb5000005242156433        0.0198     -0.4615           1.0   

mode                                      Cycle                     Walks  \
date                                impact_rate recovery_rate impact_rate   
classification toid                                                         
Local Road     Osgb4000000027869139      0.0000       -1.0000      0.0000   
               Osgb4000000027869149      0.0000        0.0000      0.0000   
               Osgb4000000027876087      0.0000        0.0000     -1.0000   
               Osgb4000000027876088      0.0000        0.0000     -0.1538   
               Osgb4000000027876095     -0.8750        1.0000     -0.5000   
...                                         ...           ...         ...   
Strategic Road Osgb5000005241746776      0.2000       -0.3333      0.2857   
               Osgb5000005241746777      0.0000        0.0000      0.0000   
               Osgb5000005241864717      1.6667       -0.8750     -0.4000   
               Osgb5000005242071802      2.2500        0.4615      0.0000   
               Osgb5000005242156433     -0.4737        1.5000     -0.1951   

mode                                               Stationary                
date                                recovery_rate impact_rate recovery_rate  
classification toid                                                          
Local Road     Osgb4000000027869139        1.5000      0.0000        0.0000  
               Osgb4000000027869149        0.0000      0.0000        0.0000  
               Osgb4000000027876087        0.0000      0.0000        0.0000  
               Osgb4000000027876088       -0.7273     -0.3333        1.0000  
               Osgb4000000027876095        3.0000     -1.0000        0.0000  
...                                           ...         ...           ...  
Strategic Road Osgb50000052417467